# 3.3 Sending a PM2.5 data stream to the Kafka broker (Step-2)

This Jupyter Notebook will be used as a producer application for our Kafka workflow. We will use the CSV file generated in the previous step and send each data record as a message to the kafka broker.

In [1]:
## Import Libraries
import pandas as pd # to handle tabular data
import json # to handle data operations in json format
from confluent_kafka import Producer # Kafka producer library to enable streaming functionalities
import socket # to get network properties for kafka communication

In [2]:
def acked(err, msg):

    '''
    This function handles callback for Kafka Producer. It handles the error/success messages
    '''
    
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg.value()), str(err)))
    else:
        print("Message produced: %s" % (str(msg.value())))

The next section runs in the following order:

1. Initialise variables: CSV filepath and topic name
2. Topic: Every Kafka message should be associated to a kafka topic, it can be named anything
3. conf: Init Kafka server with IP:Port

In [3]:
topic = "pm25_stream"
p_key = "../data/sample_multilocation.csv"
#p_key = "Chunk2024"

In [4]:
### START: AVOID MAKING CHANGES ###

### This section defines how to connect to the Kafka server the configuration
### of which is defined in the docker file. Changing this section including port numbers
### or other variables can result in broken kafka connection and the data cannot be streamed then

conf = {'bootstrap.servers': "kafka:9093", 'client.id': socket.gethostname()}
producer = Producer(conf)

### END: AVOID MAKING CHANGES ###

In [5]:
# Read CSV using Pandas
df = pd.read_csv(p_key)

In [6]:
# Init For Loop for number of records in CSV file
for i in range(df.shape[0]): ## df.shape returns dimension of the dataframe (rows, columns)

    result = {} ## Init Dict
    result[df.loc[i,'value']] = [df.loc[i,'lat'], df.loc[i,'lon'], str(df.loc[i,'day']), df.loc[i,'boxId']]

    '''
    Format of result JSON:

    {
        'pm25_value_1': [lat, lon, day, boxId]
    }
    '''

    # Store as JSON as Kafka supports JSON transmission as standard
    result = json.dumps(result)
    
    ## Key is optional, used to categorize messages by partition, in this case all messages get the same partition name. 
    ## Is Mostly used for scalability 

    producer.produce(topic, key=p_key, value=result, callback=acked) ## Message is stored in broker memory at this point

    ## Complete the sending of message from buffer (broker memory) and get an acknowledgement
    producer.flush()

Message produced: b'{"11.232186012449652": [51.96422, 7.645218, "2024-01-14T11:00:00.000Z", "591f578c51d34600116a8ea5"]}'
Message produced: b'{"4.779399926820344": [51.96422, 7.645218, "2024-01-15T12:00:00.000Z", "591f578c51d34600116a8ea5"]}'
Message produced: b'{"6.492572264910355": [51.96422, 7.645218, "2024-01-16T13:00:00.000Z", "591f578c51d34600116a8ea5"]}'
Message produced: b'{"9.893489392831016": [51.96422, 7.645218, "2024-01-17T14:00:00.000Z", "591f578c51d34600116a8ea5"]}'
Message produced: b'{"10.07761680550525": [51.96422, 7.645218, "2024-01-18T15:00:00.000Z", "591f578c51d34600116a8ea5"]}'
Message produced: b'{"16.394211287988423": [51.96422, 7.645218, "2024-01-19T16:00:00.000Z", "591f578c51d34600116a8ea5"]}'
Message produced: b'{"5.122278481012659": [51.96422, 7.645218, "2024-01-20T17:00:00.000Z", "591f578c51d34600116a8ea5"]}'
Message produced: b'{"31.14744842562432": [51.96422, 7.645218, "2024-01-21T18:00:00.000Z", "591f578c51d34600116a8ea5"]}'
Message produced: b'{"25.80354

# KAFKA Producer

At this point you should have successfully streamed the downloaded data into the kafka broker where it is currently stored in a temporary file.

#### END STEP - 2